# Predicting Health Code Violations in Boston
### Capstone Milestone Report
Roy Wright  
June 21, 2016  
  

## The Problem

We want to find restaurants that are likely to fail an unannounced health inspection &mdash; that is, we will estimate the probability that a health code violation would be found at a given food service establishment, if it were chosen to be inspected, without prior notice, right now. 


## The Client

The City of Boston, like many others, conducts health inspections of food service establishments in a largely random pattern of visits. With a model for predicting where and when health code violations might be expected, the inspections can be carried out in a targeted way. This could make more efficient use of inspectors' time and, more importantly, increase the chances of detecting violations.

## Datasets
### Health Inspections
Detailed records of health inspections throughout Boston, from 2006 to present, can be obtained from [data.cityofboston.gov](https://data.cityofboston.gov/Health/Food-Establishment-Inspections/qndu-wx8w). Each inspection record includes the following pieces of information, among others: 
* Food service establishment's name
* Category and type
 - category "FS" stands for "Eating & Drinking"
 - category "FT" stands for "Eating & Drinking w/ Take Out"
 - category "MFW" stands for "Mobile Food Walk On"
 - category "RF" stands for "Retail Food"
* The result of the inspection
 - "HE_Fail" when an establishment's first inspection (or first yearly inspection) is unsatisfactory
 - "HE_FailExt" when an establishment is reinspected within a few weeks after a failure, and fails again
 - "HE_Pass" when an establishment passes an inspection with no violations (either its first yearly inspection or a reinspection a few weeks after failure)
 - "HE_Filed" for the first inspection of an opening establishment
 - other result codes are much less common
* The date of the inspection
* Summary and description of each violation found, along with its severity, measured as \* or \*\* or \*\*\*
* Written comments
* Business address 
* Latitude/longitude coordinates (sometimes)

Below, we load the data and make the following adjustments:
* Cut off any records before August 1, 2011, because one of our other datasets does not extend back farther than this
* Separate the latitude/longitude coordinates for later use
* Process address information into a form that could more easily be used to look up latitude/longitude coordinates where they are missing

A portion of some randomly-selected lines of the inspections data is shown.

In [2]:
import pandas as pd
from numpy import nan
import numpy as np
import scipy.stats as stats
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
inspections = pd.read_csv(
    'data/Food_Establishment_Inspections.csv', 
    usecols = ['BusinessName', 'LICENSECAT',
       'DESCRIPT', 'RESULT', 'RESULTDTTM', 'Violation', 'ViolLevel',
       'ViolDesc', 'VIOLDTTM', 'ViolStatus', 'StatusDate', 'Comments', 'Address', 'City', 'State', 'Zip', 'Location'
    ],
    parse_dates = ['RESULTDTTM', 'VIOLDTTM', 'StatusDate'], 
    infer_datetime_format = True
)
inspections.columns = [
    'name',           # business name
    'category',       # business category ("FS", "FT", "RS", or "MFW")
    'type',           # more verbal description of business category 
    'result',         # result of the inspection (12 different possibilities)
    'result_date',    # date of inspection
    'violation',      # code for violation type
    'level',          # violation severity (can be * or ** or ***)
    'description',    # verbal description of violation
    'violation_date', # generally (or always?) the same as result_date
    'status',         # simply pass or fail
    'status_date',    # generally a day or so later than result_date, but often not given
    'comments',       # free-form text comments
    'address',
    'city',
    'state',
    'zip',
    'location'        # (latitude, longitude)
]
inspections['result_date'] = inspections.result_date.apply(lambda x: x.date())
inspections = inspections[
    (inspections.result_date.astype(str) >= '2011-08-01') 
]
inspections['location'] = inspections.location.str.strip('()')
inspections[['latitude','longitude']] = inspections['location'].apply(lambda x: pd.Series(str(x).split(',')))
inspections['latitude'] = pd.to_numeric(inspections.latitude, errors='coerce')
inspections['longitude'] = pd.to_numeric(inspections.longitude, errors='coerce')
inspections[['address']] = (
    inspections.address.fillna(inspections.name) + ', ' + inspections.city.fillna('Boston') + ', ' 
    + inspections.state.fillna('MA') + (' 0'+inspections.zip.fillna(0).astype(int).astype(str)).replace(' 00','')
)
inspections = inspections.drop(['city', 'state', 'zip', 'location'], 1)
inspections.sample(8)[['name', 'category', 'type', 'result', 'result_date', 'level', 'description', 'comments']]

,name,category,type,result,result_date,level,description,comments
300764,CAFE PODIMA,FT,Eating & Drinking w/ Take Out,HE_Pass,2016-01-21,*,Hand Cleaner Drying Tissue Signage,No paper towel dispenser at front counter hand...
326494,SHERATON BOSTON (APROPOS/R.S./CAFE),FS,Eating & Drinking,HE_Fail,2016-02-11,*,Food Protection,Observed raw shell eggs stored with deli slice...
179013,Brick House Pizza & Grille,FT,Eating & Drinking w/ Take Out,HE_Filed,2012-04-03,*,Food Contact Surfaces Design,Plastic containers being used to dispense flow...
303396,SAME OLD PLACE INC.,FS,Eating & Drinking,HE_Fail,2013-09-03,*,Clean Cloths Hair Restraint,NaN
233731,Lilly's Pasta & Subs,FT,Eating & Drinking w/ Take Out,HE_Pass,2012-02-10,*,Food Container Labels,Provide labels for all food containers and sua...
169317,CHECKMATE CAFE,FT,Eating & Drinking w/ Take Out,HE_Pass,2015-08-20,*,Wiping Cloths Clean Sanitize,4-101.16 Provide commercial grade scrubber for...
303609,Boston Shawarma,FS,Eating & Drinking,HE_Filed,2012-04-04,*,Improper Storage of Re-usable Utensils,store all pots and pans upside down to reduce ...
403337,BEE'S KNEES,RF,Retail Food,HE_Pass,2015-08-24,**,Insects Rodents Animals,Remove heavy fly activity at the dumpster area...


### Condensed Violations Records
[Yelp.com](https://www.yelp.com/dataset_challenge/drivendata) published academic datasets for the 2015 "Keeping it Fresh: Predict Restaurant Inspections" contest, hosted by [DrivenData.org](https://www.drivendata.org/competitions/5/page/17/).

At the heart of these datasets is a list of inspection results. Each row gives the inspection date, a "restaurant ID" number for the establishment inspected, and tallies of the violations found at each severity level (\*, \*\*, and \*\*\*). Another dataset gives a summary of Yelp-provided information for each business, including its name, address, "Yelp ID" number (which is distinct from its "restaurant ID"), number of Yelp reviews available, average rating, and latitude/longitude coordinates. Lastly, a table is provided for matching Yelp ID's to restaurant ID's.

Below, we load these datasets. Then, after a slight reshaping of the ID matching table, we use that table to combine the other datasets into one. As before, we cut off any records before August 1, 2011. A few randomly-selected lines of the combined dataset is shown.

In [11]:
violations = pd.read_csv(
    'data/AllViolations.csv', 
    parse_dates = ['date'], 
    infer_datetime_format = True,
    index_col = 0
)

In [12]:
import json
from pandas.io.json import json_normalize

with open('data/yelp_academic_dataset_business.json', 'r') as f:
    data = f.readlines()
data = map(lambda x: x.rstrip(), data)
data_json_str = '[' + (',').join(data) + ']'

business = pd.read_json(data_json_str)
business = business[[
    'name',
    'full_address',
    'city',
    'business_id',
    'review_count', 
    'stars',
    'latitude',
    'longitude'
]]
business.columns = [
    'name',     
    'address',
    'city',
    'yelp_id',
    'reviews',
    'rating',
    'latitude',
    'longitude'
]

%reset_selective -f data_json_str

In [13]:
idtable = pd.read_csv(
    'data/restaurant_ids_to_yelp_ids.csv'
)
idtable = pd.concat([
        idtable[['restaurant_id','yelp_id_0']].rename(columns = {'yelp_id_0':'yelp_id'}),
        idtable[idtable.yelp_id_1.notnull()][['restaurant_id','yelp_id_1']].rename(columns = {'yelp_id_1':'yelp_id'}),
        idtable[idtable.yelp_id_2.notnull()][['restaurant_id','yelp_id_2']].rename(columns = {'yelp_id_2':'yelp_id'}),
        idtable[idtable.yelp_id_3.notnull()][['restaurant_id','yelp_id_3']].rename(columns = {'yelp_id_3':'yelp_id'})
], ignore_index = True)

In [14]:
violations = violations.merge(
    idtable, 
    on = 'restaurant_id'
).merge(
    business, 
    on = 'yelp_id'
).drop(['yelp_id', 'address', 'city'], 1)
violations = violations[violations.date >= '2011-08-01'].sort_values('date')
violations.sample(5)

,date,restaurant_id,*,**,***,name,reviews,rating,latitude,longitude
5911,2014-07-17,1m3M8pEB,0,0,0,The Warren Tavern,211,4.0,42.374226,-71.063110
7062,2015-05-19,1m3MJDoB,8,0,3,Spring Street Cafe,33,3.0,42.272129,-71.171897
35400,2013-01-18,wmo7R73q,8,0,0,Derne Street Deli,31,4.0,42.359477,-71.063477
18774,2012-12-30,Y1EmQ7Ew,0,0,0,Toscano Restaurant,294,4.0,42.357408,-71.069847
13428,2013-01-16,XJ3rZZ3R,0,0,0,Starbucks,15,3.0,42.348542,-71.082583


### Comparison of the Inspections Datasets

We have two separate datasets that provide records of past health inspections. To compare these datasets, let's take a look at the contents of each one for a single day. Below, we display the contents for August 12, 2014. This date is chosen mostly arbitrarily.

In [7]:
inspections[inspections.result_date.astype(str) == '2014-08-12'][[
        'name', 'category', 'result', 'result_date', 'level', 'description', 'comments','latitude','longitude'
    ]].sort_values('name')

,name,category,result,result_date,level,description,comments,latitude,longitude
363499,A C Farm Market,RF,HE_Pass,2014-08-12,*,Non-Food Contact Surfaces,Repair rusty shelving below prep table.,42.302302,-71.059800
393035,A C Farm Market,RF,HE_Pass,2014-08-12,*,Food Protection,Discontinue store vegetables in stagnant wate...,42.302302,-71.059800
367015,A C Farm Market,RF,HE_Pass,2014-08-12,**,Insects Rodents Animals,Evidence of fruit flies provide exterminators ...,42.302302,-71.059800
375200,A C Farm Market,RF,HE_Pass,2014-08-12,*,Dishwashng Facilities,Replace missing sink plugs.,42.302302,-71.059800
377256,A C Farm Market,RF,HE_Pass,2014-08-12,*,Food Container Labels,Provide labels for all packaged foods.,42.302302,-71.059800
144937,Choice's by Au Bon Pain,FT,HE_Fail,2014-08-12,*,Non-Food Contact Surfaces,bakery/replace worn door gasket to 1 door reac...,NaN,NaN
149089,Choice's by Au Bon Pain,FT,HE_Fail,2014-08-12,***,Cold Holding,salad bar/carrot and raisin salad 49 degrees/c...,NaN,NaN
148702,Choice's by Au Bon Pain,FT,HE_Fail,2014-08-12,*,Equipment Thermometers,front line/ 2 door drawer/provide internal the...,NaN,NaN
144049,Choice's by Au Bon Pain,FT,HE_Fail,2014-08-12,*,Non-Food Contact Surfaces Clean,salad bar/clean interior cabinets,NaN,NaN
146524,Choice's by Au Bon Pain,FT,HE_Fail,2014-08-12,*,Improper Maintenance of Floors,replace damaged floor tile in front of proof b...,NaN,NaN


In [8]:
violations[violations.date == '2014-08-12'].drop(['reviews','rating'], 1).sort_values('name')

,date,restaurant_id,*,**,***,name,latitude,longitude
8004,2014-08-12,we39j9ok,4,0,0,Dunkin' Donuts,42.349264,-71.042474
20155,2014-08-12,lnORdd3N,0,0,0,Dunkin' Donuts,42.356527,-71.053353
33990,2014-08-12,KAoK8ZOg,0,0,0,Fóumami,42.356039,-71.053455
7757,2014-08-12,0ZEDGWOD,9,0,0,Great Chef Chinese Food Day Square,42.379525,-71.027940
1423,2014-08-12,njoZ1D3r,3,0,1,Kantin,42.352744,-71.125447
2566,2014-08-12,eVOBLr3j,1,0,1,Lollicup,42.352444,-71.125403
35075,2014-08-12,B1oXNlEV,11,1,3,Max Brenner,42.349491,-71.080588
25898,2014-08-12,B1oX4boV,4,0,1,Samurai Kuang Eatery,42.355741,-71.058335
28452,2014-08-12,8xExZeo0,4,1,1,South Boston Chinese Restaurant,42.336483,-71.047309


At a glance, we can see that the inspection records maintained by the City of Boston are more detailed than those provided by Yelp. It is easier to compare the two if we regroup the city's records in a way more like Yelp's version, as shown below.

Note that some businesses present in the city's records do not appear in Yelp's data. In fact, it seems that no business of the "retail food" type appears in Yelp's data. On the other hand, every inspection listed in Yelp's data can be seen represented in the city's data, although not always clearly. For example, the first Dunkin' Donuts inspection listed above, with four 1-star violations and no 2- or 3-star violations, is listed at the bottom of the table below, with the name "WORLD TRADE CENTER" (which is actually the location of that particular Dunkin' Donuts). But aside from that, each of the other inspections shown above can be found without much difficulty in the table below.

In [9]:
inspections[
    inspections.result_date.astype(str) == '2014-08-12'\
].fillna('none').groupby([
        'name','level','result','type'
    ]).count()[['status']]

status
name                                           level result     type                                 
A C Farm Market                                *     HE_Pass    Retail Food                         4
                                               **    HE_Pass    Retail Food                         1
Choice's by Au Bon Pain                        *     HE_Fail    Eating & Drinking w/ Take Out       4
                                               ***   HE_Fail    Eating & Drinking w/ Take Out       1
City Sports                                    *     HE_Fail    Retail Food                         2
Convenient Corner                              *     HE_Filed   Retail Food                         1
DUNKIN DONUTS(FRANKLIN)                        none  HE_Pass    Eating & Drinking w/ Take Out       1
Foumami                                        none  HE_Pass    Eating & Drinking w/ Take Out       1
Great Chef                                     *     HE_Pass    Eating & Drinking w/ Take Out       9
Harvard Stadium Stand No. 2                    none  HE_Pass    Eating & Drinking w/ Take Out       1
KANTIN                                         *     HE_Fail    Eating & Drinking w/ Take Out       3
                                               ***   HE_Fail    Eating & Drinking w/ Take Out       1
LOLLICUP TEA ZONE                              *     HE_Fail    Eating & Drinking w/ Take Out       1
                                               ***   HE_Fail    Eating & Drinking w/ Take Out       1
Max Brenner                                    *     HE_Fail    Eating & Drinking                  11
                                               **    HE_Fail    Eating & Drinking                   1
                                               ***   HE_Fail    Eating & Drinking                   3
Mcdonalds (Warren St.)                         *     HE_Fail    Eating & Drinking w/ Take Out       2
                                               ***   HE_Fail    Eating & Drinking w/ Take Out       1
Mud House                                      *     HE_Pass    Eating & Drinking                   2
                                               ***   HE_Pass    Eating & Drinking                   1
Nevilles Bakery & Restaurant                   *     HE_FailExt Eating & Drinking w/ Take Out       8
                                               **    HE_FailExt Eating & Drinking w/ Take Out       1
Panera Bread Bakery Cafe                       *     HE_Pass    Eating & Drinking                   5
                                               ***   HE_Pass    Eating & Drinking                   1
Pho Viets                                      *     HE_Filed   Eating & Drinking w/ Take Out       1
Pollos A La Brasa Beto's                       *     HE_Filed   Eating & Drinking w/ Take Out       7
Popeyes Chicken & Biscuit                      *     HE_Fail    Eating & Drinking w/ Take Out       2
SOUTH BOSTON CHINESE                           *     HE_Pass    Eating & Drinking w/ Take Out       4
                                               **    HE_Pass    Eating & Drinking w/ Take Out       1
                                               ***   HE_Pass    Eating & Drinking w/ Take Out       1
Samurai Kuang Eatery                           *     HE_Pass    Eating & Drinking w/ Take Out       4
                                               ***   HE_Pass    Eating & Drinking w/ Take Out       1
Savory Food Truck No.  3 (P-33-235)            *     HE_Filed   Mobile Food Walk On                 1
Shanghai Social Club                           *     HE_Pass    Eating & Drinking                   2
                                               **    HE_Pass    Eating & Drinking                   1
                                               ***   HE_Pass    Eating & Drinking                   1
Shaw's Supermarket No. 586                     *     HE_Fail    Retail Food                         6
               

There seem to be some mistakes in Yelp's condensed records. To see why, note that in the city records above, the inspection of Samurai Kuang Eatery on August 12 is marked as passing. For that inspection, four 1-star and one 3-star violations are noted, but this is only because that inspection was a follow-up to an inspection from one week before, which found those violations. This can be seen in the excerpt of city data below. Unfortunately, the Yelp data treats each of these inspections identically, marking four 1-star and one 3-star violations for each inspection, which is quite misleading. This mistaken double-entry of health violations happens frequently throughout the Yelp dataset. 

We will see later that this issue, once we make an appropriate correction for it, will not at all impede the central purpose of this project.

In [10]:
inspections[
    (inspections.name.str.startswith('Samurai Kuang')) & 
    (inspections.result_date.astype(str) >= '2014-08') &
    (inspections.result_date.astype(str) < '2015') 
][[
        'name', 'result', 'result_date', 'level', 'description', 'comments'
    ]].sort_values('result_date')

,name,result,result_date,level,description,comments
63245,Samurai Kuang Eatery,HE_Fail,2014-08-05,*,Non-Food Contact Surfaces,Back door opened without screen / Provide scr...
63322,Samurai Kuang Eatery,HE_Fail,2014-08-05,***,Cold Holding,Sushi grade salmon 51F White fish 50F / Provi...
63343,Samurai Kuang Eatery,HE_Fail,2014-08-05,*,Improper Maintenance of Walls/Ceilings,Hood vents with visible grease build up / Clea...
63378,Samurai Kuang Eatery,HE_Fail,2014-08-05,*,Improper Maintenance of Floors,Floors under cookline around handsink heavily...
63480,Samurai Kuang Eatery,HE_Fail,2014-08-05,*,Equipment Thermometers,Dish machine gauge is broken / Repair.
63080,Samurai Kuang Eatery,HE_Pass,2014-08-12,*,Improper Maintenance of Walls/Ceilings,Hood vents with visible grease build up / Clea...
63148,Samurai Kuang Eatery,HE_Pass,2014-08-12,*,Non-Food Contact Surfaces,Back door opened without screen / Provide scr...
63204,Samurai Kuang Eatery,HE_Pass,2014-08-12,*,Improper Maintenance of Floors,Floors under cookline around handsink heavily...
63352,Samurai Kuang Eatery,HE_Pass,2014-08-12,*,Equipment Thermometers,Dish machine gauge is broken / Repair.
63410,Samurai Kuang Eatery,HE_Pass,2014-08-12,***,Cold Holding,Sushi grade salmon 51F White fish 50F / Provi...


In [11]:
violations[
    (violations.name.str.startswith('Samurai Kuang')) & 
    (violations.date.astype(str) >= '2014-08') &
    (violations.date.astype(str) < '2015') 
].drop(['reviews','rating','latitude','longitude'], 1).sort_values('date')

,date,restaurant_id,*,**,***,name
25897,2014-08-05,B1oX4boV,4,0,1,Samurai Kuang Eatery
25898,2014-08-12,B1oX4boV,4,0,1,Samurai Kuang Eatery
25899,2014-12-02,B1oX4boV,0,0,2,Samurai Kuang Eatery
25900,2014-12-03,B1oX4boV,0,0,2,Samurai Kuang Eatery


In light of the excerpts and discussion above, we can now consider the advantages of each dataset of inspection results. 

The City of Boston's dataset is kept continually up-to-date, with new results being entered as they occur, while Yelp's data ends in mid 2015. Boston's dataset is also more complete in the sense that each violation is categorized in much finer detail than a simple a three-level severity measurement. This information could possibly be used in a predictive model &mdash; for example, perhaps some specific types of violations are more prone to repeat than others are.

Yelp's dataset was developed with the express support of the City of Boston, working toward a purpose very similar to the purpose of the present project. This dataset provides latitude/longitude coordinates for every inspection location listed. Also, Yelp's inspection records are designed for easy matching to another extensive dataset (not yet shown here) containing the text of Yelp reviews for each establishment.

Given enough time and resources, the limitations of either dataset could be corrected. However, I believe that for our present purpose, the advantages of Yelp's inspection dataset far outweigh its limitations.

### Service Request Calls

Boston residents are able to dial "311" and report public property issues such as rodent sightings, unsanitary conditions, streetlight outages, and so forth. Records of these 311 service requests, from July 2011 to present, are available from [data.cityofboston.gov](https://data.cityofboston.gov/City-Services/311-Service-Requests/awu8-dc52).

Each call record includes the following relevant information: 
* The date the complaint was made
* The date the issue was resolved
* Various descriptions of the nature of the complaint
* Latitude/longitude coordinates of the issue

Below, we load the data and display some randomly-selected lines from it.

In [12]:
services = pd.read_csv(
    'data/311__Service_Requests.csv', 
    usecols = [
       'OPEN_DT', 'CLOSED_DT', 'CASE_STATUS', 'CLOSURE_REASON', 'CASE_TITLE', 'SUBJECT', 'REASON',
       'TYPE', 'LATITUDE', 'LONGITUDE'
    ],
    parse_dates=['OPEN_DT', 'CLOSED_DT'], 
    infer_datetime_format = True
)
services.columns = [
    'open',         # date the complaint was registered
    'closed',       # date the complaint was resolved
    'status',       # open or closed
    'closure',      # details of how the complaint was closed (NaN if open)
    'title',        # description of the issue
    'subject',      # category of the issue (e.g. public works, civil rights, animal control, etc.)
    'reason',       # more specific category (often very similar to the "title")
    'type',         # usually identical to the "title" 
    'latitude',      
    'longitude'      
]
services = services.sort_values(['open', 'reason'])
services.sample(8)

,open,closed,status,closure,title,subject,reason,type,latitude,longitude
316304,2011-09-29 07:37:52,2011-09-29 15:07:59,Closed,Case Closed Case Resolved pothole patched on 0...,Highway Maintenance,Public Works Department,Highway Maintenance,Highway Maintenance,42.3131,-71.1238
669093,2013-08-19 15:02:08,2013-08-19 15:12:11,Closed,Case Closed Case Resolved The tree was pruned...,Tree Maintenance Requests,Parks & Recreation Department,Trees,Tree Maintenance Requests,42.3088,-71.1046
525745,2013-09-23 14:57:19,2013-09-26 21:48:15,Closed,Case Closed Bulk Item Automation,Schedule a Bulk Item Pickup,Public Works Department,Sanitation,Schedule a Bulk Item Pickup,42.3014,-71.0568
75745,2015-08-05 13:47:08,2015-08-10 16:37:44,Closed,Case Closed. Closed date : 2015-08-10 16:37:44...,Schedule Bulk Item Pickup,Public Works Department,Sanitation,Schedule a Bulk Item Pickup SS,42.3495,-71.0893
201423,2015-11-14 14:06:18,2015-11-15 08:03:01,Closed,Case Closed. Closed date : 2015-11-15 08:03:01...,Pick up Dead Animal,Public Works Department,Street Cleaning,Pick up Dead Animal,42.3499,-71.0897
115340,2011-08-30 07:58:05,2011-08-30 08:10:22,Closed,Case Closed Case Scheduled Items have been sch...,Schedule a Bulk Item Pickup,Public Works Department,Sanitation,Schedule a Bulk Item Pickup,42.3597,-71.0693
584670,2015-09-16 21:59:00,NaT,Open,NaN,Tree Maintenance Requests,Parks & Recreation Department,Trees,Tree Maintenance Requests,42.2838,-71.1206
453203,2014-07-30 11:08:14,2014-08-04 16:37:46,Closed,Case Closed Bulk Item Automation,Schedule a Bulk Item Pickup,Public Works Department,Sanitation,Schedule a Bulk Item Pickup,42.3004,-71.0807


Each complaint is described, often redundantly, by a "title," a "subject," a "reason," and a "type." In the dataset, there are 7837 different titles, 18 different subjects, 61 different reasons, and 215 different types. We will use "reasons" as a natural way to categorize complaints, because they strike a balance between being overly specific (as in the thousands of different "titles") and not being descriptive enough (like the handful of vague "subjects").

For future use and convenience, we will store the names of the most commonly used "reasons." These are shown below.

In [13]:
reason = services.reason.value_counts().head(45).reset_index()['index']
reason

0                            Sanitation
1                   Highway Maintenance
2                       Street Cleaning
3                         Street Lights
4                       Signs & Signals
5                               Housing
6                             Recycling
7      Enforcement & Abandoned Vehicles
8                              Building
9                                 Trees
10                             Graffiti
11          Employee & General Comments
12               Environmental Services
13            Park Maintenance & Safety
14                     Code Enforcement
15    Administrative & General Requests
16                        Animal Issues
17                         Notification
18                               Health
19                         Call Inquiry
20                           Operations
21     Traffic Management & Engineering
22                               Survey
23                           Disability
24                           Catchbasin


## Extracting Features from the Data

To develop a model for predicting a food service establishment's likely outcome (pass or fail) from an unannounced health inspection, we will make use of two main types of information:
* Conditions in the area near the business in the recent past, as drawn from the service call request data outlined above.
* Customer impressions of the business, as given in Yelp ratings and written reviews.

The Yelp dataset of inspection results already has some basic features drawn from customer impressions. These are the number of reviews, and the average customer rating for each establishment. These features are flawed in a few ways, perhaps most importantly in that they are summarized over all time (rather than just up to the date of inspection), so we will eventually replace them with better features drawn from the deeper Yelp review data available. 

In the code below, we extract potential features related to conditions in the area near each inspected business in the recent past. For each of the common service call "reasons" discussed before, we count the number of times that particular issue has been reported near the inspected business, between 5 and 15 days before the inspection. Nearness is judged here in terms of distance in latitude/longitude coordinates; roughly speaking, we are counting any reports that have occurred within about 3 miles of the inspected business. *(The time window and distance used here were chosen after some light experimentation, but can be further adjusted for better predictive ability.)*

For each business inspected, we also measure the length of time that has passed since the previous inspection.

In [15]:
pd.options.mode.chained_assignment = None
for j in range(45):
    violations[reason[j]] = nan

violations['delay'] = nan
    
for row in range(len(violations)):
    c = (violations.latitude.iloc[row], violations.longitude.iloc[row])
    ri = violations.restaurant_id.iloc[row]
    t = violations.date.iloc[row]
    tlast = violations[(violations.date <= t - timedelta(days = 1)) & (violations.restaurant_id == ri)].date.max()
    violations['delay'].iloc[row] = t - tlast
    s = services[
            (services.open < t - timedelta(days = 5)) &
            (services.open > t - timedelta(days = 15)) &
            ((services.latitude - c[0])**2 + (services.longitude - c[1])**2 < .0025)
    ]
    for j in range(45):
        violations.iloc[row,j+10] = sum(s.reason == reason[j])
        
violations['delay'] = (violations.delay) / (timedelta(days=1))
violations['delay'] = violations.delay.astype(float)

Below, for demonstration purposes, we display the information for inspections that occurred on July 22, 2014. With these we can illustrate the meaning of the features extracted above. For example, an inspection occurred at Penguin Pizza that day. This inspection was failed, since violations were found. Between 5 and 15 days prior to this inspection, there were 499 complaints related to "sanitation" in the area around Pengin Pizza, 325 complaints related to "highway maintenance," 104 related  to "street cleaning," and so on. It had been 208 days since Penguin Pizza's last inspection. 

Farther down in the table below, we see that Rebecca's Cafe was inspected the same day, and passed (no violations). 5 to 15 days prior to this inspection, there were 452 complaints related to "sanitation" in the area around Rebecca's Cafe, 297 complaints related to "highway maintenance," and so forth. 

Observe that the "delay" column, which marks the number of days that have passed since each establishment was last inspected, contains numbers of two very distinct magnitudes. This reflects the fact that many inspections are follow-ups to recent failed inspections. As shown before, the violations information given for these follow-ups is often not accurate.

In [16]:
violations[violations.date == '2014-07-22'].drop(['restaurant_id','latitude','longitude'], 1)

,date,*,**,***,name,reviews,rating,Sanitation,Highway Maintenance,Street Cleaning,...,Water Issues,Alert Boston,Volunteer & Corporate Groups,Generic Noise Disturbance,Pothole,Boston Bikes,Parking Complaints,Cemetery,Office of The Parking Clerk,delay
9951,2014-07-22,22,2,12,New Saigon Sandwich,257,4.0,523.0,305.0,125.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,186.0
1014,2014-07-22,8,0,1,Penguin Pizza,386,4.0,499.0,325.0,104.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,208.0
20419,2014-07-22,3,0,0,Starbucks,17,4.0,565.0,359.0,135.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,148.0
802,2014-07-22,13,0,0,My Thai Vegan Cafe,421,4.0,522.0,306.0,124.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,204.0
27566,2014-07-22,9,2,1,Al Dente Restaurant,529,4.0,381.0,287.0,90.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,140.0
12117,2014-07-22,5,0,0,Restaurante Cesaria,30,4.0,662.0,339.0,196.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0
12574,2014-07-22,6,2,2,Anh Hong,113,4.0,549.0,220.0,186.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,166.0
11026,2014-07-22,10,2,2,Teriyaki House,189,3.5,555.0,315.0,152.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,11.0
11783,2014-07-22,35,2,11,Xinh Xinh,608,4.0,522.0,306.0,124.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0
33509,2014-07-22,7,2,2,Benevento's,194,3.5,380.0,287.0,90.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,140.0


To get an early sense of the relationship between the features extracted above and the prevalence of inspection failures, we may examine the correlation coefficients between each feature and each of the three severity levels, as follows.

In [18]:
violations[['*','**','***'] + list(reason[0:45]) + ['reviews', 'rating', 'delay']].corr().iloc[0:3,3:]

,Sanitation,Highway Maintenance,Street Cleaning,Street Lights,Signs & Signals,Housing,Recycling,Enforcement & Abandoned Vehicles,Building,Trees,...,Volunteer & Corporate Groups,Generic Noise Disturbance,Pothole,Boston Bikes,Parking Complaints,Cemetery,Office of The Parking Clerk,reviews,rating,delay
*,0.108451,0.154236,-0.010561,-0.009663,0.089541,0.074555,0.131223,0.105753,0.112984,0.105726,...,0.097123,0.033182,0.107008,0.018981,0.020321,0.052117,0.053696,0.073952,-0.015761,-0.157320
**,0.075268,0.075548,-0.010698,-0.000860,0.054662,0.049962,0.080476,0.058439,0.062179,0.074257,...,0.069487,-0.004128,0.041324,0.003882,0.018903,0.049752,0.024959,0.049787,-0.029659,-0.115727
***,0.047084,0.073965,-0.021544,-0.038988,0.046897,0.039782,0.063104,0.046075,0.062928,0.059040,...,0.050193,0.017577,0.072613,0.012584,0.002113,0.023598,0.018101,0.024487,-0.013495,-0.122651


For some features, the correlation with violations is essentially nonexistent, but for others there is a weak correlation. Many of these correlations are statistically significant. For example, here are the *p*-values from tests of correlation for each of the first 10 features above.

In [19]:
pd.DataFrame({
        'reason':[reason[k] for k in range(0,10)], 
        'p-value (*)':[stats.pearsonr(violations['*'], violations[reason[k]])[1] for k in range(0,10)],
        'p-value (**)':[stats.pearsonr(violations['**'], violations[reason[k]])[1] for k in range(0,10)],
        'p-value (***)':[stats.pearsonr(violations['***'], violations[reason[k]])[1] for k in range(0,10)]
    },index=range(0,10)).set_index('reason')

,p-value (*),p-value (**),p-value (***)
reason,,,
Sanitation,4.375221e-47,1.742967e-23,4.325481e-10
Highway Maintenance,6.478502e-94,1.196805e-23,9.862901e-23
Street Cleaning,1.617405e-01,1.563345e-01,4.306347e-03
Street Lights,2.004226e-01,9.092561e-01,2.370376e-07
Signs & Signals,1.375353e-32,4.237129e-13,5.070446e-10
Housing,4.517745e-23,3.500459e-11,1.342927e-07
Recycling,2.801139e-68,1.264433e-26,5.799759e-17
Enforcement & Abandoned Vehicles,7.591063e-45,9.236586e-15,1.009582e-09
Building,5.615606e-51,1.633028e-16,7.070723e-17


Returning to the table of correlation coefficients above, on the far right side of the table we see that the average customer rating for establishments is somewhat negatively correlated with the number of violations found, which is not surprising. However, we alse see a stronger negative correlation between the "delay" (the length of time that has passed since the last inspection) and the number of violations found. This indicates that the more time has passed since the last inspection, the fewer violations are found &mdash; or in other words, establishments are most likely to be in violation of health codes shortly after their last inspection. This seems nonsensical; indeed, it is merely an artifact of the "mistaken double-entry of health violations" that was pointed out before.

Fortunately, the goal of this project gives us ample justification for ignoring any data from follow-up inspections. We seek a model for identifying businesses that are likely to fail an **unannounced** health inspection. Any business that has already failed a recent inspection will necessarily be inspected again, regardless of any prediction we might make about the outcome of that follow-up. So in the context of this project, there is no value to the City of Boston in predicting the outcomes of follow-up inspections. Therefore in the remaining analysis, we will exclude any data from follow-up inspections, focusing solely on the outcomes of inspections that were not expected in advance.

Below, we calculate the correlation coefficients between each feature and each of the three severity levels, but now omitting follow-up inspections. In most cases where there was correlation before, it is now stronger.

In [20]:
violations[violations.delay > 45][['*','**','***'] + list(reason[0:45]) + ['reviews', 'rating', 'delay']].corr().iloc[0:3,3:]

,Sanitation,Highway Maintenance,Street Cleaning,Street Lights,Signs & Signals,Housing,Recycling,Enforcement & Abandoned Vehicles,Building,Trees,...,Volunteer & Corporate Groups,Generic Noise Disturbance,Pothole,Boston Bikes,Parking Complaints,Cemetery,Office of The Parking Clerk,reviews,rating,delay
*,0.139246,0.195804,-0.007455,-0.003573,0.119235,0.099797,0.157217,0.135949,0.157053,0.177941,...,0.105806,0.033354,0.117984,0.036522,0.042902,0.083069,0.059529,0.066402,-0.020065,0.016038
**,0.092148,0.110467,0.004545,-0.007807,0.084856,0.067311,0.098524,0.076487,0.093464,0.112405,...,0.078871,0.001561,0.058061,0.018840,0.025826,0.066199,0.027245,0.044643,-0.040314,0.012783
***,0.083102,0.125820,-0.020036,-0.029770,0.081129,0.066252,0.087781,0.081670,0.100093,0.117873,...,0.062146,0.012172,0.091903,0.023659,0.019698,0.042357,0.023871,0.030237,-0.019569,0.014625


## Some Hypothesis Tests

Since the question we wish to answer &mdash; will a given establishment fail an unannounced health inspection or not? &mdash; is binary in nature, let's test the influence that our environmental features have on the proportion of failed inspections. For each kind of common issue that may be reported through service calls, we can test the following hypotheses.

**Null hypothesis:** Establishments where the issue has been reported recently and nearby fail health inspections at the *same* rate as establishments where the issue has not been reported recently and nearby.

**Alternative hypothesis:** Establishments where the issue has been reported recently and nearby fail health inspections at a *different* rate than establishments where the issue has not been reported recently and nearby.

Building upon [Allen Downey's ``HypothesisTest`` class](https://github.com/AllenDowney/CompStats/blob/master/hypothesis.ipynb), the following code provides a method to test for a difference in proportions.

In [6]:
class HypothesisTest(object):
    """Represents a hypothesis test."""

    def __init__(self, data):
        """Initializes.

        data: data in whatever form is relevant
        """
        self.data = data
        self.MakeModel()
        self.actual = self.TestStatistic(data)
        self.test_stats = None

    def PValue(self, iters=1000):
        """Computes the distribution of the test statistic and p-value.

        iters: number of iterations

        returns: float p-value
        """
        self.test_stats = np.array([self.TestStatistic(self.RunModel()) 
                                       for _ in range(iters)])

        count = sum(self.test_stats >= self.actual)
        return count / iters

    def MaxTestStat(self):
        """Returns the largest test statistic seen during simulations.
        """
        return max(self.test_stats)

    def PlotHist(self, label=None):
        """Draws a Cdf with vertical lines at the observed test stat.
        """
        ys, xs, patches = pyplot.hist(ht.test_stats, color=COLOR4)
        pyplot.vlines(self.actual, 0, max(ys), linewidth=3, color='0.8')
        pyplot.xlabel('test statistic')
        pyplot.ylabel('count')

    def TestStatistic(self, data):
        """Computes the test statistic.

        data: data in whatever form is relevant        
        """
        raise UnimplementedMethodException()

    def MakeModel(self):
        """Build a model of the null hypothesis.
        """
        pass

    def RunModel(self):
        """Run the model of the null hypothesis.

        returns: simulated data
        """
        raise UnimplementedMethodException()


class DiffPropsPermute(HypothesisTest):
    """Tests a difference in proportions by permutation."""

    def TestStatistic(self, data):
        """Computes the test statistic.

        data: data in whatever form is relevant        
        """
        group1, group2 = data
        test_stat = abs(sum(group1)/len(group1) - sum(group2)/len(group2))
        return test_stat

    def MakeModel(self):
        """Build a model of the null hypothesis.
        """
        group1, group2 = self.data
        self.n, self.m = len(group1), len(group2)
        self.pool = np.hstack((group1, group2))

    def RunModel(self):
        """Run the model of the null hypothesis.

        returns: simulated data
        """
        np.random.shuffle(self.pool)
        data = self.pool[:self.n], self.pool[self.n:]
        return data

For example, we can test whether businesses that are near the sites of recent complaints related to "environmental services" are more likely than other businesses to fail a health inspection. (Note that "environmental services" complaints largely consist of rodent sightings.)

We perform the test below.

In [22]:
set1 = violations[(violations.delay > 45) & (violations['Environmental Services'] == 0)][['*','**','***']] 
set2 = violations[(violations.delay > 45) & (violations['Environmental Services'] > 0)][['*','**','***']]
data1 = (set1['*'] + set1['**'] + set1['***'] > 0)
data2 = (set2['*'] + set2['**'] + set2['***'] > 0)
ht_data = (data1, data2)
ht = DiffPropsPermute(ht_data)
(ht.TestStatistic(ht_data), ht.PValue(iters=1000))

(0.036418197095186677, 0.78100000000000003)

There is no evidence to indicate a difference in failure rates between businesses with *zero* nearby environmental services complaints and businesses with *more than zero* such complaints. However, businesses with exactly *zero* of these complaints nearby are quite rare; the vast majority of inspected businesses are near at least 10 of them. 

Is there a difference between businesses that are near very few (less than 10) recent environmental services complaints and others? Here is the test:

In [23]:
set1 = violations[(violations.delay > 45) & (violations['Environmental Services'] < 10)][['*','**','***']]
set2 = violations[(violations.delay > 45) & (violations['Environmental Services'] >= 10)][['*','**','***']]
data1 = (set1['*'] + set1['**'] + set1['***'] > 0)
data2 = (set2['*'] + set2['**'] + set2['***'] > 0)
ht_data = (data1, data2)
ht = DiffPropsPermute(ht_data)
(ht.TestStatistic(ht_data), ht.PValue(iters=1000))

(0.21239383082277097, 0.0)

The *p*-value here is indistinguishable from zero. There is a statistically significant difference in the inspection failure rates of businesses that are near at least 10 recent environmental services complaints. In our data, these establishments fail inspections at a rate 21% higher than others.

As another example, there is a significant difference in failure rates between businesses that are near very few (less than 100) recent "sanitation" complaints and other businesses:

In [24]:
set1 = violations[(violations.delay > 45) & (violations['Sanitation'] < 100)][['*','**','***']]
set2 = violations[(violations.delay > 45) & (violations['Sanitation'] >= 100)][['*','**','***']]
data1 = (set1['*'] + set1['**'] + set1['***'] > 0)
data2 = (set2['*'] + set2['**'] + set2['***'] > 0)
ht_data = (data1, data2)
ht = DiffPropsPermute(ht_data)
(ht.TestStatistic(ht_data), ht.PValue(iters=1000))

(0.25831625477195092, 0.0)

Similar results can be found for many of the other environmental features, which suggests that these features &mdash; after some refinement &mdash; could be useful in developing a model to fulfill our stated purpose.

As for features derived from customer experiences, we have further work to do. At the present time we may ask, is there a difference in failure rates between businesses with a high average customer rating and others? It is not clear what we should use as a cutoff value between "high" ratings and others, so let's see the results for every possible rating cutoff:

In [25]:
for c in np.arange(1.5, 5, .5):
    set1 = violations[(violations.delay > 45) & (violations.rating <= c)][['*','**','***']]
    set2 = violations[(violations.delay > 45) & (violations.rating > c)][['*','**','***']]
    data1 = (set1['*'] + set1['**'] + set1['***'] > 0)
    data2 = (set2['*'] + set2['**'] + set2['***'] > 0)
    ht_data = (data1, data2)
    ht = DiffPropsPermute(ht_data)
    print(
        'With cutoff ' + str(c) + 
        ', the p-value is ' + str(ht.PValue(iters=1000)) + 
        ' and the actual difference in proportions is ' + str(ht.TestStatistic(ht_data))
    )

With cutoff 1.5, the p-value is 0.7 and the actual difference in proportions is 0.0271862170973
With cutoff 2.0, the p-value is 0.735 and the actual difference in proportions is 0.00988379783767
With cutoff 2.5, the p-value is 0.266 and the actual difference in proportions is 0.0173930022535
With cutoff 3.0, the p-value is 0.782 and the actual difference in proportions is 0.00309852093128
With cutoff 3.5, the p-value is 0.036 and the actual difference in proportions is 0.0230290541312
With cutoff 4.0, the p-value is 0.0 and the actual difference in proportions is 0.116197340704
With cutoff 4.5, the p-value is 0.513 and the actual difference in proportions is 0.136516314779


As noted before, the average customer rating feature is deeply flawed, but even if we put that fact aside for a moment, the prospects of using it to help distinguish between likely passes and likely failures is not promising. We only find statistically significant results when using a rating cutoff of 4.0, and then the actual difference in failure rates is only about 12%. 

We will have more success after extracting some much more detailed features from the text of Yelp reviews, as mentioned previously.

## Some preliminary work with Yelp reviews text

Loading the dataset of Yelp reviews text, and aligning it for reference with the violations dataset:

In [ ]:
import json
from pandas.io.json import json_normalize

reviews = pd.DataFrame()
with open('data/yelp_academic_dataset_review.json', 'r') as f:
    for line in f:
        df = pd.read_json(line).set_index('review_id').iloc[[0]].drop(['user_id','votes','type'], axis=1)
        reviews = reviews.append(df)
reviews = reviews.rename(
    columns = {'business_id':'yelp_id'}
).merge(
    idtable, 
    on = 'yelp_id'
).drop(
    'yelp_id', 
    axis = 1
)
reviews[['date']] = pd.to_datetime(reviews.date)

In [46]:
reviews

,review_id,date,stars,text,restaurant_id
0,OeT5kgUOe3vcN7H6ImVmZQ,2005-08-26,3,This is a pretty typical cafe. The sandwiches...,N6Ok7qOx
1,qq3zF2dDUh3EjMDuKBqhEA,2005-11-23,3,I agree with other reviewers - this is a prett...,N6Ok7qOx
2,i3eQTINJXe3WUmyIpvhE9w,2005-11-23,3,"Decent enough food, but very overpriced. Just ...",N6Ok7qOx
3,cnAvoSxsMtyuPEmB9wJ0Nw,2006-02-24,5,The muffins are great...esp the blueberry! I ...,N6Ok7qOx
4,bs4VNLZUHi0Rh7Lz1BRKyw,2007-09-06,3,"Well, well, well, look at me reviewing the res...",N6Ok7qOx
5,F1v9N06fzLCU9M2R3NEGCw,2007-11-11,4,The only place downtown where you can get away...,N6Ok7qOx
6,zQH071b6x9g1ZHbhJnaNKw,2005-12-11,4,This is the place I like to go for deli sandwi...,p03824Om
7,NR33cz48RMMYfT79MaDxKQ,2006-04-19,4,Delicato is a great place for lunch on the go ...,p03824Om
8,70bfgIrBIZYu3HmgAGtAEQ,2008-05-06,4,Delicato is a surprisingly tasty place for bei...,p03824Om
9,AUmMDjcDYfCTLmNbRv2dQA,2008-08-02,4,Did you ever notice that no matter where you g...,p03824Om


For each inspection, find any Yelp reviews written in the past year, take the mean of the numerical ratings, and compile (i.e. concatenate) the text:

In [62]:
pd.options.mode.chained_assignment = None

violations['delay'] = nan
    
for row in range(len(violations)):
    ri = violations.restaurant_id.iloc[row]
    t = violations.date.iloc[row]
    tlast = violations[(violations.date <= t - timedelta(days = 1)) & (violations.restaurant_id == ri)].date.max()
    violations['delay'].iloc[row] = t - tlast
    r = reviews[
        (reviews.restaurant_id == ri) &
        (reviews.date <= t - timedelta(days = 1)) &
        (reviews.date >= t - timedelta(days = 365))
    ]
    violations['rating'].iloc[row] = r.stars.mean()
    violations['reviews'].iloc[row] = r['text'].str.cat(sep=' ')
    
        
violations['delay'] = (violations.delay) / (timedelta(days=1))
violations['delay'] = violations.delay.astype(float)

In [63]:
violations

,date,restaurant_id,*,**,***,name,reviews,rating,latitude,longitude,delay
15836,2011-08-01,NbE1Bk3J,8,1,0,Crazy Dough's Pizza,Manger une pizza a pas chers c est la place qu...,3.750000,42.346810,-71.088960,NaN
1401,2011-08-01,7RO5vjEq,7,1,0,Arirang House,Buffet joints aren't usually where you go to g...,2.125000,42.346246,-71.087049,NaN
25218,2011-08-01,V430D43B,1,0,2,The Florentine Cafe,Was told it was a 5 min wait for a table. They...,3.289474,42.364263,-71.053833,NaN
34217,2011-08-01,Y1EmaVEw,7,0,1,Q Restaurant,Dropped in with a friend during one of their s...,3.464286,42.351813,-71.062679,NaN
27586,2011-08-01,6Wo2Nyo9,11,2,1,Caribe,,NaN,42.315208,-71.066018,NaN
33235,2011-08-01,8x3zgYok,2,0,1,Au Bon Pain,I like Au Bon Pain very much. Whether it's th...,1.833333,42.349664,-71.070235,NaN
6368,2011-08-01,8x3zx2Ok,5,0,2,Boston Marriott Copley Place,We stayed at this hotel for 3 nights for the f...,3.523810,42.347022,-71.079289,NaN
21033,2011-08-01,qN3gvnEA,5,0,1,Taiwan Café,It's a love/hate relationship. There are times...,3.629630,42.351503,-71.060239,NaN
21523,2011-08-01,VpoG57Er,1,0,0,Papa Gino's Pizzeria,,NaN,42.255668,-71.123935,NaN
6157,2011-08-01,dj3dlN39,1,0,2,Sakkio Japan,Get the teriyaki chicken with EXTRA chicken! ...,4.000000,42.355682,-71.060278,NaN


Create and try to optimize the parameters for a naive Bayes classifier:

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB

def make_xy(violations, vectorizer=None):
    if vectorizer is None:
        vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(violations.reviews)
    X = X.tocsc()
    y = (violations['*'] + violations['**'] + violations['***'] > 0).values.astype(np.int)
    return X, y

In [4]:
violations = violations[violations.delay > 45]

bestscore = -1
bestmd = -1
bestMd = -1
besta = -1
for md in (0.0, 0.01, 0.05, 0.1, 0.2, 0.4):
    for Md in (0.5, 0.6, 0.7, 0.8, 0.9, 1.0):
        for a in (1, 5, 8, 12, 20):

            X, y = make_xy(violations, vectorizer=CountVectorizer(min_df = md, max_df = Md, ngram_range = (1,1)))

            clf = MultinomialNB(alpha = a)
            Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=.8)
            clf.fit(Xtrain, ytrain)
            thisscore = clf.score(Xtest, ytest)
            print(
                'min: ' + str(md) +
                ', max: ' + str(Md) +
                ', alpha: ' + str(a) +
                ', train: ' + str(clf.score(Xtrain, ytrain)) +
                ', test: ' + str(thisscore)
            )
            if thisscore > bestscore:
                bestscore = thisscore
                bestmd = md
                bestMd = Md
                besta = a

min: 0.0, max: 0.5, alpha: 1, train: 0.746854403835, test: 0.599401197605
min: 0.0, max: 0.5, alpha: 5, train: 0.688735769922, test: 0.625748502994
min: 0.0, max: 0.5, alpha: 8, train: 0.655931695626, test: 0.635928143713
min: 0.0, max: 0.5, alpha: 12, train: 0.645146794488, test: 0.641916167665
min: 0.0, max: 0.5, alpha: 20, train: 0.639604553625, test: 0.631736526946
min: 0.0, max: 0.6, alpha: 1, train: 0.750599161174, test: 0.607784431138
min: 0.0, max: 0.6, alpha: 5, train: 0.686488915518, test: 0.651497005988
min: 0.0, max: 0.6, alpha: 8, train: 0.657878969443, test: 0.648502994012
min: 0.0, max: 0.6, alpha: 12, train: 0.643199520671, test: 0.641317365269
min: 0.0, max: 0.6, alpha: 20, train: 0.635260635111, test: 0.650299401198
min: 0.0, max: 0.7, alpha: 1, train: 0.746554823247, test: 0.578443113772
min: 0.0, max: 0.7, alpha: 5, train: 0.686788496105, test: 0.631137724551
min: 0.0, max: 0.7, alpha: 8, train: 0.663720790893, test: 0.634730538922
min: 0.0, max: 0.7, alpha: 12, tra

In [5]:
bestmd, bestMd, besta, bestscore

(0.0, 1.0, 8, 0.65449101796407183)

Re-create the classifier with the best parameters found (accuracy 0.654), and list some of the most predictive words:

In [29]:
vectorizer=CountVectorizer(min_df = 0.0, max_df = 1.0)
X, y = make_xy(violations, vectorizer)

clf = MultinomialNB(alpha = 8)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=.8)
clf.fit(Xtrain, ytrain)

MultinomialNB(alpha=8, class_prior=None, fit_prior=True)

In [30]:
words = np.array(vectorizer.get_feature_names())

probs = np.zeros(len(words))

for i in range(len(words)):
    x = np.zeros(len(words))
    x[i] = 1
    probs[i] = clf.predict_log_proba(x.reshape(1,-1))[0,0]


ind = np.argsort(probs)

bad_words = words[ind[:30]]
good_words = words[ind[-30:]]

bad_prob = probs[ind[:30]]
good_prob = probs[ind[-30:]]

print( "Bad words\t     P(fail | word)")
for w, p in zip(bad_words, bad_prob):
    print( "%20s" % w, "%0.2f" % (1 - np.exp(p)))
    
print( "Good words\t     P(fail | word)")
for w, p in zip(good_words, good_prob):
    print( "%20s" % w, "%0.2f" % (1 - np.exp(p)))

Bad words	     P(fail | word)
             trident 0.97
               doyle 0.96
              sonsie 0.96
           bookstore 0.96
             penguin 0.95
             kashmir 0.95
              durgin 0.95
                masa 0.92
                olio 0.92
              grotto 0.92
             orinoco 0.91
              haggis 0.91
               aglio 0.91
            merengue 0.90
           dominican 0.90
                frik 0.89
              fleuri 0.88
            bleacher 0.88
             trolley 0.88
             typhoon 0.88
               arepa 0.88
                  5n 0.87
              snappy 0.87
              parker 0.87
              barlow 0.87
              zocalo 0.86
               haley 0.86
              napkin 0.86
            scottish 0.86
               lucia 0.86
Good words	     P(fail | word)
              schlow 0.23
                ober 0.23
               skits 0.23
               locke 0.22
              lorenz 0.22
                 ufo 0.22
   